In [171]:
import numpy as np
import csv
import scipy.optimize as opt

##### This part is just test data, the function is below

#Choosing a yearly risk-free rate
rf = 0.03;
#Monthly data needs monthly rf rate
rfm = rf/12;

# first let's read the data we need to test on
#stock price data
stock_dat = 'MIE377_Project.csv'
stock_tab = []
with open(stock_dat, 'r') as csvfile:
    stockreader = csv.reader(csvfile, delimiter = ' ')
    for row in stockreader:
        stock_tab.append(row)

stock_tab2 = []
for i in range(1, len(stock_tab)):
    stock_tab2.append(stock_tab[i][0].split(','))        
        
#convert stock data to float type
for i in range(0, len(stock_tab2)):
    #stock_tab2[i][0] = int(stock_tab2[i][0])
    for j in range(0, len(stock_tab2[i])):
        stock_tab2[i][j] = float(stock_tab2[i][j])
        
periods = len(stock_tab2) - 1
stocks = len(stock_tab2[0]) - 1
#first get numpy matrix of price data
price = np.array([stock_tab2[i][0:(stocks + 1)] for i in range(0, periods + 1)],\
                 dtype = np.float64)
#get return data

#get return as (P[i] - P[i-1])/P[i-1]
rets = (price[0:periods,:-1] - price[1:(periods+1),:-1])/price[1:(periods+1),:-1]
#now get mean returns
#axis 0 pertains to columns, which is what I am looking for (i.e. mean return of each stock)
mu = np.mean(rets, axis = 0)

#get covariance matrix
Q = np.cov(rets, rowvar = False)        

In [205]:
import numpy as np
import csv
import scipy.optimize as opt

def mvo(Q, mu, mu_goal, rf):
    #Q is the covariance matrix
    #mu is the list of asset returns (row vector)
    #mu_goal and rf are the return goals
    def variance(x,data):
        Q, mu, mu_goal, rf = data
        #X is a row-vector
        #Q is the covariance matrix
        z = np.array(x)
        z = z[:-1] #Remove the risk free asset
        var = (1/2)*(np.matmul(z,np.matmul(Q,np.transpose(z))))
        return var

    #Constraint Definitions
    def ret_goal(x, data):
        Q, mu, mu_goal, rf = data
        #x and mu are row vectors
        z = np.array(x)
        #add the risk free return
        z_mu = np.append(np.array(mu), rf)
        val = np.matmul(z,np.transpose(z_mu)) - mu_goal
        return val

    def budget(x, data):
        Q, mu, mu_goal, rf = data
        return (np.sum(x)-1)
    
    #Arg structure
    data = [Q, mu, mu_goal, rf]
    
    #Constraint structure
    cons = [{'type':'eq', 'fun': budget, 'args': (data,)},
        {'type':'ineq', 'fun': ret_goal, 'args': (data,)}]
    
    
    #Initial guess construction
    x0 = np.zeros(len(mu)+1)
    x0[0] = 1
    
    #No short selling condition
    bounds = [(0, np.inf) for _ in x0]
    
    #Optimization
    res = opt.minimize(variance, x0, args=(data,), method='SLSQP',
                       constraints=cons, options={'ftol': 1e-9, 'disp': True}, bounds = bounds)
    
    
    port_var = res.fun
    port_weights = res.x
    port_ret = np.matmul(np.array(res.x),np.transpose(np.append(mu,rf)))
    
    return((port_weights,port_var,port_ret))


Optimization terminated successfully.    (Exit mode 0)
            Current function value: 6.889261698469497e-05
            Iterations: 8
            Function evaluations: 88
            Gradient evaluations: 8


(array([6.03069620e-01, 5.42101086e-19, 1.00918011e-01, 1.04565209e-01,
        0.00000000e+00, 1.23224061e-01, 6.81762655e-02, 9.48676901e-20,
        4.68334854e-05]), 6.889261698469497e-05, 0.005000000000157543)